# Analisis de posibilidad de join entre datos sernapesca

### Import libreries

In [15]:
import logging
import subprocess
import os   
import pandas as pd
import seaborn as sns

In [16]:
%%time
reporte_plantas_raw = pd.read_excel("../data/datos_sernapesca/Reporte de Plantas.xlsx")
ingresos_raw = pd.read_excel("../data/datos_sernapesca/Ingresos 2021-2022.xlsx")
RNA_raw = pd.read_excel("../data/datos_sernapesca/RNA.xlsx")

CPU times: total: 41.9 s
Wall time: 41.9 s


### Filtrando por región y tipo de producto

In [17]:
# Se filtra por region, no es necesario hacer filtro por producto ya que solo se encuentra 
# info disponible para moluscos
reporte_plantas_filtered = reporte_plantas_raw[(reporte_plantas_raw['Región Bodega'] == 'X REGION') |
                                        (reporte_plantas_raw['Región Agente'] == 'X REGION')]

# Se filtra sólo por tipo de especie = molusco, ya que este dataset ya era sólo de la décima región
ingresos_filtered = ingresos_raw[ingresos_raw['Tipo Especie'] == 'MOLUSCOS']

# El registro nacional de acuicultura de sernapesca (RNA) se encuentra para todo chile, por lo que se filtra
# para la décima región. También se filtra por aquellos centros que se encuentran inscritos.
RNA_filtered = RNA_raw[(RNA_raw['Región'] == 10) & (RNA_raw['Categoria'] == 'INSCRITO')]


In [18]:
shape_reportes =  reporte_plantas_filtered.shape
shape_ingresos =  ingresos_filtered.shape
shape_RNA =  RNA_filtered.shape
print(f'''El dataset de reportes filtrado tiene una forma {shape_reportes}, 
        mientras que el de ingresos es de la forma {shape_ingresos} y 
        el de RNA es de {shape_RNA}''')

El dataset de reportes filtrado tiene una forma (52011, 35), 
        mientras que el de ingresos es de la forma (7998, 22) y 
        el de RNA es de (2338, 18)


Se tienen 3 datasets y queremos saber si es que es posible hacer join entre los 3. Para esto y en primera instancia es necesario definir los campos que podrían ser utilizados para el join. 

* Por parte de la tabla de ingresos, código de agente parece ser una buena opción y también código del último proovedor.

* Para el caso de la tabla de reportes, existe el codigo de centro y el codigo de origen, los cuales podrían servir.

* Finalmente la tabbla de RNA contiene el código del centro de cultivo acuícola.

In [23]:
#ingresos_filtered

RNA_list_CCAA = RNA_filtered['Codigo Centro'].unique().tolist()
reporte_plantas_list_origen = reporte_plantas_filtered['Código Origen'].unique().tolist()

In [51]:
results = {}
for i in RNA_list_CCAA:
    results[i] = reporte_plantas_list_origen.count(i) 

In [52]:
df_RNA_reporte_plantas = pd.DataFrame.from_dict(results,orient='index', columns=['count'])
#type(df_RNA_reporte_plantas['count'])
ccaa_match_with_reporte = df_RNA_reporte_plantas[df_RNA_reporte_plantas['count'] > 0]
ccaa_match_with_reporte


,count
100022,1
100242,1
100339,1
101000,1
101011,1
...,...
104342,1
104347,1
104349,1
104366,1
